<a href="https://colab.research.google.com/github/zihoonkim/inf_car_number/blob/main/Tensor_softmax_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import tensorflow as tf
import numpy as np

# 1. 가설 정의

## mnist 구성
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() #load_data()의 반환 값이 mnist의 데이터이고 numpy배열로 되어 있다
x_train, x_test = x_train.astype('float32'), x_test.astype('float32') #astype를 통해 float형으로 변환 시켜주고
x_train, x_test = x_train.reshape([-1, 784]), x_test.reshape([-1, 784]) #reshape를 통해 2차원을 1차원으로 바꿔 계산하기 쉽게 만들어
x_train, x_test = x_train / 255., x_test / 255. # 픽셀의 범위를 0에서 1사이로 만들어 
y_train, y_test = tf.one_hot(y_train, depth=10), tf.one_hot(y_test, depth=10) #원 핫 인코딩을 통해 60000개의 데이터를 10차원으로 바꿨다 60000 x 10

# api 를 이용해 batch
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train)) # 미니배치화 하기 위해 tf.data.Dataset의 from_tensor_slices를 이용
train_data = train_data.repeat().shuffle(60000).batch(100) #반복해서 섞어 100개씩 배치화
train_data_iter = iter(train_data)# 순서대로 가져오는



# tf.keras.Model 이용 소프트맥스 정의
class SoftmaxRegression(tf.keras.Model):
  def __init__(self):
    super(SoftmaxRegression, self).__init__()
    self.softmax_layer = tf.keras.layers.Dense(10,      #케라스의 하위 api인 dense를 이용해  784차원을 10차원으로 바꾼뒤
                                               activation=None, #활성화 함수는 설정하지 않고
                                               kernel_initializer='zeros', # 초기 w 값 zero로 설정
                                               bias_initializer='zeros'   # 초기 bias 값 zero로 설정
                                               )
  def call(self, x):#x는 input 값
    logits = self.softmax_layer(x) 

    return tf.nn.softmax(logits) # 출력 결과 값


# cross_entropy 손실함수 정의.
@tf.function
def cross_entropy_loss(y_pred, y):
  return tf.reduce_mean(-tf.reduce_sum(y * tf.math.log(y_pred), axis=[1]))
  

# 경사하강법 optimizer.
optimizer = tf.optimizers.SGD(0.5)

# 최적화를 위한 function을 정의합니다.
@tf.function
def train_step(model, x, y):
  with tf.GradientTape() as tape:
    y_pred = model(x)
    loss = cross_entropy_loss(y_pred, y)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables)) # 이 trainable_variables를 통해 모델 구조의 전체 list를 반환

# 모델의 정확도
@tf.function
def compute_accuracy(y_pred, y):
  correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  return accuracy # 0~ 1사이의 정확도를 반환

# SoftmaxRegression 모델을 선언합니다.
SoftmaxRegression_model = SoftmaxRegression()

# 1000번 반복을 수행하면서 최적화를 수행합니다.
for i in range(1000):
  batch_xs, batch_ys = next(train_data_iter) ## batch_xs = (100, 784), batch_ys = one_hot
  train_step(SoftmaxRegression_model, batch_xs, batch_ys)

# 학습이 끝나면 학습된 모델의 정확도를 출력합니다.
print("정확도(Accuracy): %f" % compute_accuracy(SoftmaxRegression_model(x_test), y_test)) 




정확도(Accuracy): 0.917900
